In [19]:
import pandas as pd
import string
import nltk
nltk.download('stopwords')
from collections import Counter
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import math
import numpy as np
import os

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/aaditya11/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
# def docPreProcessing(filepath):
#     stopwords_dict = Counter(stopwords.words('english'))

#     file = open(filepath, 'r', errors='replace')
#     lines = file.readlines()
    
#     # print(filepath)

#     stopwords_dict = Counter(stopwords.words('english'))
    
    
#     doctext = ""
#     for line in lines:
#         line = line.translate(str.maketrans('', '', string.punctuation)).strip().lower()
#         line = ' '.join([word for word in line.split() if word not in stopwords_dict])
#         doctext += line
        
#     doctextList = doctext.split()
#     return doctextList

In [15]:
def docPreProcessing(filepath):
    stopwords_dict = Counter(stopwords.words('english'))


    docTextList = []
    with open(filepath, 'r', errors='replace') as file:
        for line in file:
            for word in line.split():
                word = word.translate(str.maketrans('', '', string.punctuation)).strip().lower()
                if word not in stopwords_dict and word != '':
                    docTextList.append(word)

    return docTextList

In [21]:
ps = PorterStemmer()
wordlist = docPreProcessing(r'../testset/sci.space/59497')

for word in wordlist:
    print(word + " " + ps.stem(word))

# docPreProcessing(r'../testset/sci.space/59497')

ettealcsnorg ettealcsnorg
eric eric
h h
taylor taylor
subject subject
gravity graviti
waves wave
predicting predict
gravity graviti
wave wave
quantization quantiz
cosmic cosmic
noise nois
article articl
c4kvjf4qowellsfcaus c4kvjf4qowellsfcau
metareswellsfcaus metareswellsfcau
tom tom
van van
flandern flandern
writes write
crb7qkelvinseasvirginiaedu crb7qkelvinseasvirginiaedu
cameron cameron
randale randal
bass bass
writes write
brucescottlaunchpaduncedu brucescottlaunchpaduncedu
bruce bruce
scott scott
writes write
existence exist
undefined undefin
unless unless
synonymous synonym
observable observ
physics physic
crb crb
dong dong
dong dong
dong dong
hear hear
deathknell deathknel
string string
theory theori
agree agre
add add
dark dark
matter matter
quarks quark
lot lot
unobservable unobserv
purely pure
theoretical theoret
constructs construct
physics physic
list list
including includ
omnipresent omnipres
black black
holes hole
bruce bruce
argue argu
existence exist
inferred infer
the

In [4]:
def tf(wordlist):
    # global tfcount
    # tfcount += 1
    wordmap = {}
    
    for word in wordlist:
        if word in wordmap:
            wordmap[word] += 1
        else:
            wordmap[word] = 1
            
    
    numWords = len(wordlist)
    
    for word in wordmap:
        wordmap[word] = wordmap[word] / numWords
    
    return wordmap
        


In [5]:
# def tfidfMapBuilder(doclist):
#     # termset = set()

#     # for doc in doclist:
#         # termset.update(set(docPreProcessing(doc)))

#     # df_columns = doclist.copy()
#     df_columns = []
#     df_columns.append('df')
#     df_columns.extend(doclist)
#     # print(df_columns)

#     df = pd.DataFrame(data=0, index = [], columns=df_columns)

#     for doc in doclist:
#         wordmap = tf(docPreProcessing(doc))
#         for term in wordmap:
#             if term not in df.index:
#                 new_row = pd.DataFrame(data=0, index=[term], columns=df_columns)
#                 df = pd.concat([df, new_row])
#             df.loc[term, doc] = wordmap[term]
#             df.loc[term, 'df'] += 1

#     # print(df)

#     # N = len(doclist)
#     # df['df'] = df['df'].apply(lambda x: math.log(N/x))

#     return df

#     # print(len(termset))

In [6]:

def tfidfMapBuilder(doclist):
    termset = set()

    for doc in doclist:
        termset.update(set(tf(docPreProcessing(doc)).keys()))

    df_columns = doclist.copy()
    df_columns.append('df')
    print(df_columns)

    df = pd.DataFrame(0, index=list(termset), columns=df_columns)

    for doc in doclist:
        wordmap = tf(docPreProcessing(doc))
        for term in wordmap:
            df.at[term, doc] = wordmap[term]
            df.at[term, 'df'] += 1

    # print(df)

    N = len(doclist)
    # df['df'] = df['df'].apply(lambda x: math.log(N/x))

    return df

    # print(len(termset))

In [7]:
def mapToMatrix(df, N):
    df['df'] = df['df'].apply(lambda x: math.log(N/x))

    for index, row in df.iterrows():
        term_df = row['df']
        row = row.apply(lambda x: x * term_df)
    
    df = df.drop(columns=['df'], axis=1)

    return df

In [8]:
def cosineSimilarityScore(q, D):
    q_mag = np.sqrt(q.dot(q))
    
    qT = np.reshape(q, (1, q.shape[0]))
    qTD = np.matmul(qT, D).reshape((D.shape[1], ))
    
    D_mags = np.sqrt(np.sum(D*D, axis=0))
    divisors = q_mag * D_mags
    cos_thetas = np.divide(qTD, divisors)
    
    cos_thetas = np.clip(cos_thetas, -1, 1)
    
    scores = np.arccos(cos_thetas)
    
    return scores
    

In [9]:
def fileCollector(path):
    filelist = []
    
    for (root, dirs, files) in os.walk(path, topdown=True):
        for file in files:
            filelist.append(os.path.join(root, file))
            
            if len(filelist) == 50:
                return filelist

    return filelist

In [10]:
doclist = []

doclist.append('query.txt')

doclist.extend(fileCollector(r'../testset/sci.space'))

tfidfDataFrame = mapToMatrix(tfidfMapBuilder(doclist), len(doclist))

# tfidfDataFrame
# tfidfMapBuilder.to_csv('tfidf.csv')

# tfidfMap = tfidfMapBuilder(doclist)

# import sqlite3
# database = "../example.sqlite"
# conn = sqlite3.connect(database)
# tfidfMap.to_sql(name='Users', con=conn, if_exists='replace', index=False)
# conn.close()

# print("saved to database!!!!!!!!!!!!!!!!")
# 
# cnx = sqlite3.connect(database)
# tfidfMap = pd.read_sql_query("SELECT * FROM Users", cnx)

# print(tfidfMap.to_numpy())

# tfidfDataFrame = mapToMatrix(tfidfMap, len(doclist))


# # tfidfMap.to_sql('tfidf', con=engine, if_exists='replace')


doc_tfidfMatrix = tfidfDataFrame.loc[:, tfidfDataFrame.columns != 'query.txt'].to_numpy().round(decimals=4)
query_vector = tfidfDataFrame.loc[:, "query.txt"].to_numpy().round(decimals=4)

['query.txt', '../testset/sci.space/60777', '../testset/sci.space/61312', '../testset/sci.space/61023', '../testset/sci.space/61090', '../testset/sci.space/61145', '../testset/sci.space/61245', '../testset/sci.space/61042', '../testset/sci.space/61433', '../testset/sci.space/61064', '../testset/sci.space/60978', '../testset/sci.space/60894', '../testset/sci.space/60910', '../testset/sci.space/60785', '../testset/sci.space/60252', '../testset/sci.space/61035', '../testset/sci.space/60888', '../testset/sci.space/61186', '../testset/sci.space/60253', '../testset/sci.space/61138', '../testset/sci.space/62477', '../testset/sci.space/60808', '../testset/sci.space/61157', '../testset/sci.space/61492', '../testset/sci.space/61515', '../testset/sci.space/60845', '../testset/sci.space/60896', '../testset/sci.space/61445', '../testset/sci.space/61036', '../testset/sci.space/61551', '../testset/sci.space/61320', '../testset/sci.space/60916', '../testset/sci.space/60170', '../testset/sci.space/6242

In [11]:
scorelist = cosineSimilarityScore(query_vector, doc_tfidfMatrix)
scoremap = {}

for index, score in enumerate(scorelist):
    scoremap[tfidfDataFrame.columns[index + 1]] = score

scoremap = dict(sorted(scoremap.items(), key=lambda item:item[1]))

for doc in list(scoremap.items())[:5]:
    print(doc)


('../testset/sci.space/60827', 1.4055722894202503)
('../testset/sci.space/61138', 1.4614697276429125)
('../testset/sci.space/61090', 1.5377657813520584)
('../testset/sci.space/60777', 1.5707963267948966)
('../testset/sci.space/61312', 1.5707963267948966)


In [36]:
doclist = []

doclist.append('query.txt')

doclist.extend(fileCollector(r'../testset/sci.space'))

print(doclist)

['query.txt', '../testset/sci.space/60777', '../testset/sci.space/61312', '../testset/sci.space/61023', '../testset/sci.space/61090', '../testset/sci.space/61145', '../testset/sci.space/61245', '../testset/sci.space/61042', '../testset/sci.space/61433', '../testset/sci.space/61064', '../testset/sci.space/60978', '../testset/sci.space/60894', '../testset/sci.space/60910', '../testset/sci.space/60785', '../testset/sci.space/60252', '../testset/sci.space/61035', '../testset/sci.space/60888', '../testset/sci.space/61186', '../testset/sci.space/60253', '../testset/sci.space/61138', '../testset/sci.space/62477', '../testset/sci.space/60808', '../testset/sci.space/61157', '../testset/sci.space/61492', '../testset/sci.space/61515', '../testset/sci.space/60845', '../testset/sci.space/60896', '../testset/sci.space/61445', '../testset/sci.space/61036', '../testset/sci.space/61551', '../testset/sci.space/61320', '../testset/sci.space/60916', '../testset/sci.space/60170', '../testset/sci.space/6242